In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

In [3]:
ptech_web = "http://ptechweb.s3-website.us-east-2.amazonaws.com"
response = get(ptech_web, headers=headers)

In [4]:
print(response.text[:500])

<!DOCTYPE html>
<!--[if IE 9 ]> <html lang="en-US" class="ie9 loading-site no-js"> <![endif]-->
<!--[if IE 8 ]> <html lang="en-US" class="ie8 loading-site no-js"> <![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><html lang="en-US" class="loading-site no-js"> <!--<![endif]-->
<head>
<meta charset="UTF-8"/>
<link rel="profile" href="http://gmpg.org/xfn/11"/>
<link rel="pingback" href="xmlrpc.php.html"/>
<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documen


In [5]:
ptech_soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
ptech_h1 = ptech_soup.find_all('h1')

In [7]:
print(ptech_h1)

[<h1>Digital Marketing Consultancy</h1>, <h1>Paid Advertising Training</h1>, <h1>International Marketing</h1>, <h1>Google Analytics Training</h1>]


In [8]:
ptech_tag = ptech_soup.find_all('h4', class_='thin-font')

for each in ptech_tag: 
    print (str(each.get_text()))

Need to understand your online marketing?
Google Adwords & Facebook Advertising
Are you looking to Export?
Learn how to reach new markets.
Get started with Analytics, understand the data


In [9]:
# setting up the lists that will form our dataframe with all the results
titles = []
metatitle = []

In [13]:
%%time

from random import randint
from time import sleep

#n_pages = 0

title_dict = {} # empty dictionary

uri = 'http://ptechweb.s3-website.us-east-2.amazonaws.com/about-us/'
suffixes = ['staff/james-pennington/index.html', 'clients/index.html'] # add the rest in here 

for suffix in suffixes: # loop through the suffixes list

    h1heading = [] # new list or empty the list to start again 
    metatitle = []
    metadesc = []
    ptech_webscrape = uri + suffix # concatenate the URL and the suffix in the current loop  
    r = get(ptech_webscrape, headers=headers)
    page_html = BeautifulSoup(r.text, 'html.parser')
    ptech_webpage = page_html.find_all('html')       
    
    if ptech_webpage != []:
        for container in ptech_webpage:
            
            # page title
            meta_title = page_html.find("meta", property="og:title")
            metatitle.append(meta_title)
            
            # meta description
            meta_desc = page_html.find("meta", property="og:description")
            metadesc.append(meta_desc)
            
            # H1
            h1name = container.find_all('h1')[0].text
            h1heading.append(h1name)
            
            
    else:
        continue

    title_dict[suffix] = h1heading, metatitle, metadesc # add to the dictionary the suffix and the list of h1's
    
    
    sleep(randint(1,3))

CPU times: user 264 ms, sys: 9.65 ms, total: 274 ms
Wall time: 4.5 s


In [14]:
print(title_dict)

{'staff/james-pennington/index.html': (['James Pennington'], [<meta content="James Pennington - ptech" property="og:title"/>], [<meta content="James has been involved in IT consultancy to SMEâs for 18+ years and has worked with over a large number of businesses in that time to help them understand and implement IT effectively within their organisations. With both a practical and strategic view of IT across all areas of the business, James works with a [...]" property="og:description"/>]), 'clients/index.html': (['Our Clients'], [<meta content="Our Clients - ptech" property="og:title"/>], [<meta content="We have delivered our services to a wide range of clients over the past 10 years, including business start-ups and Global organisations. Â What ever the type of business you’re running, we are confident that we can provided the skills and support required to take your business to the next level. Our clients, whether they be [...]" property="og:description"/>])}


In [12]:
cols = ['Title'] # columns in our xls

ptechxls = pd.DataFrame({'Title': title_dict})[cols]
ptechxls.to_excel('ptech_scrape.xls') # the name of our file